In [64]:
# Instala as bibliotecas necessárias
%pip -q install google-genai google-adk requests folium
%pip -q install tabulate
%pip -q install pandas

In [66]:
# Instala as bibliotecas necessárias
%pip -q install google-genai google-adk requests

# Importa as bibliotecas
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown, HTML
import requests
import warnings
from tabulate import tabulate
import pandas as pd


warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [67]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"



In [68]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
# Esta função simula a execução de um agente em uma sessão.
def call_agent(agent: Agent, message_text: str) -> str:

    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    print(f"🤖 Executando Agente: {agent.name}...")
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
              for part in event.content.parts:
                if part.text is not None:
                  final_response += part.text
                  final_response += "\n"

        print(f"✅ Execução do Agente {agent.name} concluída.")
    except Exception as e:
        print(f"❌ Erro durante a execução do Agente {agent.name}: {e}")
        final_response = f"Ocorreu um erro ao processar sua solicitação com o agente {agent.name}. Detalhes: {e}"

    return final_response


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def display_search_info(response):

    try:
        if response and response.candidates and response.candidates[0].grounding_metadata:
            print(f"�� Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
            if response.candidates[0].grounding_metadata.grounding_chunks:
                 print(f"�� Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks if site.web])}")
            print()
            if response.candidates[0].grounding_metadata.search_entry_point:
                 display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))
        else:
            print("ℹ️ Nenhuma informação de busca disponível para esta resposta.")
    except Exception as e:
        print(f"⚠️ Não foi possível exibir as informações de busca: {e}")


In [69]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [70]:
# --- Agente 1: Buscador de Supermercados Proximos--- #
#####################################################

def agente_supermercados_proximos(produto: str, localizacao: str) -> str:
    buscador_supermercados = Agent(
        name="agente_buscador_supermercados",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de localização de Supermercados. Sua missão é utilizar a ferramenta de busca do Google (google_search)
        para encontrar Supermercados próximos à localização fornecida pelo usuário.
        Busque por "Supermercados perto de {localizacao}" ou "Supermercados em {localizacao}".
        Para cada Supermercado encontrado nos resultados da busca, indentifique e extraia o nome, endereço e, **principalmente, o número de telefone**.
        Liste cada Supermercado encontrado com seus telefones de forma clara e sucinta.
        Caso não encontre telefones para algum Supermercado, informe o usuário
        Caso não encontre nenhum Supermercado na busca, informe o usuário.
        Sua principal missão é listar os supermercados que parecem mais relevantes ou mais próximos com base nos resultados da busca.
        """,
        description="Agente que busca Supermercados próximos e seus telefones usando Google Search.",
        tools=[google_search]
    )

    entrada_do_agente = f"Encontre Supermercados Proximos de {localizacao} que possam ter {produto}. Liste os nomes, endereços e telefones."

    resultados_busca = call_agent(buscador_supermercados, entrada_do_agente)


    return resultados_busca

In [71]:
# --- Agente 2: Comparador de Preços de Produtos --- #
########################################################

def agente_comparador_precos(produto: str) -> str:
    comparador_precos = Agent(
        name="agente_comparador_precos",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de comparação de preços de Produtos. Sua missão é usar a ferramenta de busca do Google (google_search)
        para encontrar o preço do Produto "{produto}" em diferentes Supermercados,sites de comparação de preços e em ou outras fontes confiáveis.
        Busque por "{produto} preço", "{produto} onde comprar mais barato", etc.
        Analise os resultados da busca para identificar os preços listados e as fontes (Supermercados, sites, folhetos online).
        Liste os preços encontrados, indicando a fonte (nome do Supermercado /site).
        **Identifique claramente a opção mais barata encontrada** e mencione a fonte.
        Caso não encontre as informações de preço, informe o usuário.
        """,
        description="Agente que busca e compara preços de Produtos usando Google Search.",
        tools=[google_search]
    )

    entrada_do_agente = f"Compare os preços do Produto {produto} e encontre a opção mais barata."

    resultados_comparacao = call_agent(comparador_precos, entrada_do_agente)

    return resultados_comparacao

In [80]:
# --- Fluxo Principal de Execução --- #
##########################################

print("🚀 Iniciando o Sistema de Busca de Supermercados e Produtos 🚀")

# --- Obter Input do Usuário ---
produtos_buscados = []
continuar = "sim"

while continuar.lower() == "sim":
    produto = input("❓ Digite o NOME do produto que você procura: ")
    produtos_buscados.append(produto)
    continuar = input("➕ Deseja adicionar mais um produto? (sim/não): ")

print("\nOs produtos que você procura são:")
for i, produto in enumerate(produtos_buscados):
    print(f"{i+1}. {produto}")

# Solicita a localização do usuário
localizacao_usuario = input("📍 Por favor, digite sua LOCALIZAÇÃO (ex: 'São Paulo, Jardim Paulista', 'CEP 01402-001'): ")

# Verifica se as entradas foram fornecidas
if not produtos_buscados or not localizacao_usuario:
    print("❌ Você esqueceu de digitar o nome do Produto ou sua localização!")
else:
    print(f"\nMaravilha! Vamos buscar informações sobre '{produtos_buscados} 'perto de '{localizacao_usuario}'.")

    # --- Etapa 1: Buscar Supermercados Próximos ---
    print("\n--- 🏥 Executando Agente 1 (Buscador de Supermercados) ---")
    supermercado_encontrados = agente_supermercados_proximos(produtos_buscados, localizacao_usuario)
    print("\n--- ✅ Resultado do Agente 1 (Supermercados Encontrados) ---\n")
    display(to_markdown(supermercado_encontrados))
    print("--------------------------------------------------------------")

    # --- Etapa 2: Comperando os Preços dos Produtos Informados ---
    print("\n--- 💰 Executando Agente 2 (Comparador de Preços) ---")
    precos_comparados = agente_comparador_precos(produtos_buscados)
    print("\n--- ✅ Resultado do Agente 2 (Comparação de Preços) ---\n")
    variavel1 = [precos_comparados]
    tabela_dados = list(zip(variavel1))
    cabecalhos = ["Comparação de Preço dos Produtos Informados"]
    print(tabulate(tabela_dados, headers=cabecalhos, tablefmt="fancy_grid"))
    print("--------------------------------------------------------------")

    print("\n🎉 Processo concluído!")

🚀 Iniciando o Sistema de Busca de Supermercados e Produtos 🚀
❓ Digite o NOME do produto que você procura: Arroz
➕ Deseja adicionar mais um produto? (sim/não): sim
❓ Digite o NOME do produto que você procura: óleo
➕ Deseja adicionar mais um produto? (sim/não): sim
❓ Digite o NOME do produto que você procura: açucar
➕ Deseja adicionar mais um produto? (sim/não): leite

Os produtos que você procura são:
1. Arroz
2. óleo
3. açucar
📍 Por favor, digite sua LOCALIZAÇÃO (ex: 'São Paulo, Jardim Paulista', 'CEP 01402-001'): R. Min. Viveiros de Castro, 38 - Copacabana, Rio de Janeiro - RJ, 22021-010

Maravilha! Vamos buscar informações sobre '['Arroz', 'óleo', 'açucar'] 'perto de 'R. Min. Viveiros de Castro, 38 - Copacabana, Rio de Janeiro - RJ, 22021-010'.

--- 🏥 Executando Agente 1 (Buscador de Supermercados) ---
🤖 Executando Agente: agente_buscador_supermercados...
✅ Execução do Agente agente_buscador_supermercados concluída.

--- ✅ Resultado do Agente 1 (Supermercados Encontrados) ---



> Com base nas informações encontradas, aqui estão alguns supermercados próximos ao endereço fornecido, juntamente com os detalhes disponíveis:
> 
> *   **Pão de Açúcar**
> 
>     *   Endereço: Rua Ministro Viveiros de Castro, 38 - Copacabana, Rio de Janeiro - RJ, 22021-010
>     *   Telefone: Não foi possível encontrar o número de telefone específico desta unidade nas fontes consultadas. Você pode tentar encontrar o telefone no site do Pão de Açúcar ou ligando para a central de atendimento.
> 
> *   **Supermercados Mundial**
> 
>     *   Endereço: Rua Siqueira Campos, 69, Copacabana, Rio de Janeiro
>     *   Telefone: Não foi possível encontrar o número de telefone específico desta unidade nas fontes consultadas. Você pode tentar encontrar o telefone no site do Mundial ou ligando para a central de atendimento.
> 
> *   **Supermercado Zona Sul**
> 
>     *   Várias localizações na Avenida Nossa Senhora de Copacabana e Rua Francisco Sá.
>     *   Telefone: Não foi possível encontrar o número de telefone específico desta unidade nas fontes consultadas. Você pode tentar encontrar o telefone no site do Zona Sul ou ligando para a central de atendimento.
> 
> Note que a disponibilidade de produtos como Arroz, óleo e açúcar pode variar entre os supermercados e seus estoques no momento da compra. Recomendo verificar diretamente com o supermercado se eles têm os produtos desejados em estoque.


--------------------------------------------------------------

--- 💰 Executando Agente 2 (Comparador de Preços) ---
🤖 Executando Agente: agente_comparador_precos...
✅ Execução do Agente agente_comparador_precos concluída.

--- ✅ Resultado do Agente 2 (Comparação de Preços) ---

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Comparação de Preço dos Produtos Informados                                                                                                                                                    │
╞════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ Com base nas informações encontradas, comparei os preços de Arroz, Óleo e Açúcar em diferentes supermercados e sites. Abaixo, apresen